In [1]:
use_gpu = False
use_ramdon_split = False
use_dataparallel = True

In [2]:
import os
import sys
sys.path.insert(0, '..')

if use_gpu:
    from utils.gpu_tools import *
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join([ str(obj) for obj in select_gpu(query_gpu())])

import time
import datetime
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split



torch.manual_seed(42)

IMAGE_WIDTH = {5: 15, 16:64, 20: 60, 60: 180}
IMAGE_HEIGHT = {5: 32, 16:48, 20: 64, 60: 96}  

## load data

here we choose 1993-2001 data as our training(include validation) data, the remaining will be used in testing.

In [4]:
raw_images = []
raw_label_df = []
raw_images = pickle.load(open('../crypto/data/240_16_8_images.dat', 'rb'))
raw_label_df = pd.read_feather('../crypto/data/240_16_8_labels.feather')
raw_images = np.array(raw_images).reshape((-1, IMAGE_HEIGHT[16], IMAGE_WIDTH[16]))



print(raw_images.shape)
print(raw_label_df.shape)

(156425, 48, 64)
(156425, 7)


In [28]:
# Calculate the index for 65% of the data
split_index = int(0.65 * len(raw_images))

# Split the images
images = raw_images[:split_index]
label_df = raw_label_df.iloc[:split_index]
# Split the labels
print(images.shape)
print(label_df.shape)
print(label_df.head())
print(label_df.tail())

(101676, 48, 64)
(101676, 7)
      Asset          Start_Date            End_Date  Daily_Return   Ret_24H  \
29314   BTC 2013-10-16 21:00:00 2013-10-17 20:00:00      0.024734  0.018115   
29315   BTC 2013-10-17 21:00:00 2013-10-18 20:00:00      0.034178  0.036533   
29316   BTC 2013-10-18 21:00:00 2013-10-19 20:00:00      0.099098  0.075556   
29317   BTC 2013-10-19 21:00:00 2013-10-20 20:00:00     -0.000116 -0.000117   
29318   BTC 2013-10-20 21:00:00 2013-10-21 20:00:00      0.083886  0.081870   

             Ret_8H    Market_Cap  
29314 -6.458785e-03  1.702245e+09  
29315  4.096767e-03  1.732996e+09  
29316 -2.842424e-02  1.795176e+09  
29317 -6.135681e-07  2.023532e+09  
29318  2.043606e-02  2.042262e+09  
        Asset          Start_Date            End_Date  Daily_Return   Ret_24H  \
47481     ENS 2022-12-19 15:00:00 2022-12-20 14:00:00     -0.029085 -0.036784   
135613    SYN 2022-12-19 15:00:00 2022-12-20 14:00:00     -0.039784 -0.054178   
110451    PHA 2022-12-19 15:00:00 202

## build dataset

In [29]:
from torch.utils.data import Dataset
import torch

class MyDataset(Dataset):
    
    def __init__(self, img, label):
        self.img = torch.Tensor(img.copy())
        self.label = torch.Tensor(label)
        self.len = len(img)

        # Ensure the image tensor is in the shape [batch, height, width]
        if len(self.img.shape) == 3:
            # Add a channel dimension: [batch, 1, height, width]
            self.img = self.img.unsqueeze(1)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.img[idx], self.label[idx]


Split method (not random split is recommended)

In [30]:
if not use_ramdon_split:
    train_val_ratio = 0.7
    split_idx = int(images.shape[0] * 0.7)
    train_dataset = MyDataset(images[:split_idx], (label_df.Ret_24H > 0).values[:split_idx])
    val_dataset = MyDataset(images[split_idx:], (label_df.Ret_24H > 0).values[split_idx:])
else:
    dataset = MyDataset(images, (label_df.Ret_24H > 0).values)
    train_val_ratio = 0.7
    train_dataset, val_dataset = random_split(dataset, \
        [int(dataset.len*train_val_ratio), dataset.len-int(dataset.len*train_val_ratio)], \
        generator=torch.Generator().manual_seed(42))
    del dataset

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, pin_memory=True)

In [31]:
sample_input, _ = next(iter(train_dataloader))
print(sample_input.shape)

torch.Size([128, 1, 48, 64])


## models

In [32]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.)
    elif isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

In [33]:
from models import baseline

device = 'cuda' if use_gpu else 'cpu'
export_onnx = True
net = baseline.Net2(48,64).to(device)
net.apply(init_weights)

if export_onnx:
    import torch.onnx
    x = torch.randn([1,1,48,64]).to(device)
    torch.onnx.export(net,               # model being run
                      x,                         # model input (or a tuple for multiple inputs)
                      "../cnn_baseline.onnx",   # where to save the model (can be a file or file-like object)
                      export_params=False,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=False,  # whether to execute constant folding for optimization
                      input_names = ['input_images'],   # the model's input names
                      output_names = ['output_prob'], # the model's output names
                      dynamic_axes={'input_images' : {0 : 'batch_size'},    # variable length axes
                                     'output_prob' : {0 : 'batch_size'}})


### Profiling

In [34]:
count = 0
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())
    count += parameters.numel()
print('total_parameters : {}'.format(count))

layer1.0.weight : torch.Size([64, 1, 5, 3])
layer1.0.bias : torch.Size([64])
layer1.1.weight : torch.Size([64])
layer1.1.bias : torch.Size([64])
layer2.0.weight : torch.Size([128, 64, 5, 3])
layer2.0.bias : torch.Size([128])
layer2.1.weight : torch.Size([128])
layer2.1.bias : torch.Size([128])
layer3.0.weight : torch.Size([256, 128, 5, 3])
layer3.0.bias : torch.Size([256])
layer3.1.weight : torch.Size([256])
layer3.1.bias : torch.Size([256])
fc1.1.weight : torch.Size([2, 49152])
fc1.1.bias : torch.Size([2])
total_parameters : 715010


In [35]:
from thop import profile as thop_profile

flops, params = thop_profile(net, inputs=(next(iter(train_dataloader))[0].to(device),))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPs = 37.573623808G
Params = 0.71501M


In [36]:
from torch.profiler import profile, record_function, ProfilerActivity

inputs = next(iter(train_dataloader))[0].to(device)

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        net(inputs)

prof.export_chrome_trace("../trace.json")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

c:\Code\DigitalAssets\liquid-vision\venv\lib\site-packages\torch\autograd\profiler.py:255: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         0.40%       1.669ms       100.00%     414.743ms     414.743ms             1  
                     aten::conv2d         0.01%      21.000us        69.46%     288.082ms      96.027ms             3  
                aten::convolution         0.03%     107.000us        69.46%     288.061ms      96.020ms             3  
               aten::_convolution         0.01%      49.000us        69.43%     287.954ms      95.985ms             3  
         aten::mkldnn_convolution        69.33%     287.553ms        69.42%     287.905ms      95.968ms             3  
                      aten::empty       

## train

In [37]:
def train_loop(dataloader, net, loss_fn, optimizer):
    
    running_loss = 0.0
    current = 0
    net.train()
    
    with tqdm(dataloader) as t:
        for batch, (X, y) in enumerate(t):
            X = X.to(device)
            y = y.to(device)
            y_pred = net(X)
            loss = loss_fn(y_pred, y.long())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss = (len(X) * loss.item() + running_loss * current) / (len(X) + current)
            current += len(X)
            t.set_postfix({'running_loss':running_loss})
    
    return running_loss

In [38]:
def val_loop(dataloader, net, loss_fn):

    running_loss = 0.0
    current = 0
    net.eval()
    
    with torch.no_grad():
        with tqdm(dataloader) as t:
            for batch, (X, y) in enumerate(t):
                X = X.to(device)
                y = y.to(device)
                y_pred = net(X)
                loss = loss_fn(y_pred, y.long())

                running_loss += loss.item()
                running_loss = (len(X) * running_loss + loss.item() * current) / (len(X) + current)
                current += len(X)
            
    return running_loss

In [39]:
# net = torch.load('/home/clidg/proj_2/pt/baseline_epoch_10_train_0.6865865240322523_eval_0.686580_.pt')

In [40]:
if use_gpu and use_dataparallel and 'DataParallel' not in str(type(net)):
    net = net.to(device)
    net = nn.DataParallel(net)

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-5)

start_epoch = 0
min_val_loss = 1e9
last_min_ind = -1
early_stopping_epoch = 5

from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter()

In [43]:
start_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
os.mkdir('../pt'+os.sep+start_time)
epochs = 100
for t in range(start_epoch, epochs):
    print(f"Epoch {t}\n-------------------------------")
    time.sleep(0.2)
    train_loss = train_loop(train_dataloader, net, loss_fn, optimizer)
    val_loss = val_loop(val_dataloader, net, loss_fn)
    tb.add_histogram("train_loss", train_loss, t)
    torch.save(net, '../pt'+os.sep+start_time+os.sep+'baseline_epoch_{}_train_{:5f}_val_{:5f}.pt'.format(t, train_loss, val_loss)) 
    if val_loss < min_val_loss:
        last_min_ind = t
        min_val_loss = val_loss
    elif t - last_min_ind >= early_stopping_epoch:
        break

print('Done!')
print('Best epoch: {}, val_loss: {}'.format(last_min_ind, min_val_loss))

Epoch 0
-------------------------------


100%|██████████| 120/120 [01:16<00:00,  1.57it/s]


Epoch 1
-------------------------------


100%|██████████| 120/120 [01:14<00:00,  1.60it/s]


Epoch 2
-------------------------------


...

Epoch 32
-------------------------------


100%|██████████| 120/120 [01:04<00:00,  1.86it/s]


Epoch 33
-------------------------------


100%|██████████| 120/120 [01:04<00:00,  1.87it/s]

Done!
Best epoch: 28, val_loss: 0.5040865683511881
